In [1]:
from numpy import *
from scipy import *
import sympy

In [2]:
Link_Relation=mat([[1,1,4,4,5,5,6,6,7,7,8,9,9,10,11,11,12,12,13],
                  [5,12,5,9,6,9,7,10,8,11,2,10,13,11,2,3,6,8,3],
                  [900,700,700,900,800,600,900,500,300,400,700,700,600,700,700,700,300,700,700],
                  [7,8,9,14,5,9,5,13,5,9,10,10,9,8,9,8,7,15,11]]
                 )
origin=[1,1,4,4]
destination=[2,3,2,3]
Travel_Request=[400,800,600,200]
OD_Total_Path=[]

In [3]:
def dfs(origin,index,destination,Link_Relation):   #根据深度优先进行搜索OD对之间的全部路径
    path.append(index)
    if index == destination:
        paths.append(path.copy())
        path.pop()

    else:
        next_node=[]
        for i in range(0,size(Link_Relation[0,:])):
            if Link_Relation[0,i]==index:
                next_node.append(Link_Relation[1,i])
        for item in next_node:
            if item not in path:
                dfs(origin,item,destination,Link_Relation)
        path.pop()

In [4]:
def searchsame(A,B):  #寻找两个列表是否具有相同的元素
    index=0 
    for i in A:
        for j in B:
            if i==j:
                index=j     
    return index

In [5]:
for i in range(0,size(origin)):
    path=[]
    paths=[]
    dfs(origin[i],origin[i],destination[i],Link_Relation)
    OD_Total_Path.append(paths)

In [6]:
#将各OD对路径用路段表示，输出Path_Link_Relation
Path_Link_Relation=[]
for i in OD_Total_Path:
    Path_Link_Array=zeros((shape(i)[0],shape(Link_Relation)[1]))
    for j in range(0,shape(i)[0]):
        for k in range(0,size(i[j])-1):
            index1=argwhere(array(Link_Relation[0,:])==i[j][k])[:,1]
            index2=argwhere(array(Link_Relation[1,:])==i[j][k+1])[:,1]
            index = searchsame(index1,index2)
            Path_Link_Array[j][index]=1
    Path_Link_Relation.append(Path_Link_Array)

In [7]:
shape(OD_Total_Path[3])

(6,)

——————————————————路径搜索完毕————————————————

In [8]:
#d定义变量，初始化路段流量
lamda=sympy.symbols('lamda')
x=zeros((1,shape(Link_Relation)[1]))

In [9]:
#获得当前各路径的初始阻抗
Current_Travel_Time=[]
for i in Path_Link_Relation:
    Current_Travel_Time.append((mat(i)*Link_Relation[3,:].T).T)

In [10]:
#获取相关各OD对之间无流通车辆情况下的阻抗最小路径
def find_min_path(Current_Travel_Time):
    Min_Index=[]
    for i in Current_Travel_Time:
        Min_Index.append(i.argmin())
    return Min_Index
Min_Index=find_min_path(Current_Travel_Time)

In [11]:
Min_Path=[]  #获取当前各OD对阻抗最小的路径
for i in range(0,len(Min_Index)):
    Min_Path.append(Path_Link_Relation[i][Min_Index[i]])

In [12]:
x=mat(Travel_Request)*Min_Path
Link_Flow_0=x
Initial_x=x
Time_Function=multiply(Link_Relation[3,:],mat((array(Link_Flow_0)/array(Link_Relation[2,:]))**4)*0.15+1)

In [13]:
#error_value=10
#M=0.1
count1=0
#while error_value>0.01:
e=10
    #Link_Flow_0=Initial_x
while e>10**(-4):
    count1=count1+1   
    t=multiply(Link_Relation[3,:],mat((array(Link_Flow_0)/array(Link_Relation[2,:]))**4)*0.15+1)
    Current_Travel_Time=[]
    for i in Path_Link_Relation:
        Current_Travel_Time.append((i*(t.T)).T)
    Min_Index=find_min_path(Current_Travel_Time)
    Min_Path=[] 
    for i in range(0,len(Min_Index)):
        Min_Path.append(Path_Link_Relation[i][Min_Index[i]])
    Link_Flow_1 = mat(Travel_Request)*Min_Path
    Difference_Path = Link_Flow_1-Link_Flow_0
    Link_Flow_2 = Link_Flow_0+lamda*Difference_Path
    t=multiply(Link_Relation[3,:],mat((array(Link_Flow_2)/array(Link_Relation[2,:]))**4)*0.15+1)
    f=sum(multiply(Difference_Path,t))
    value_lamda = sympy.solve(f)
    lamda1=[]
    for i in value_lamda:
        try :
            i=float(i)
            lamda1.append(i)
        except:
            continue
    for i in lamda1:
        if i>0 and i<1: 
            lamda_bechoosed = i
    print(lamda_bechoosed)
    Link_Flow_2=Link_Flow_0+multiply(lamda_bechoosed,Difference_Path)          
    e = sqrt(((Link_Flow_2-Link_Flow_0)*(Link_Flow_2-Link_Flow_0).T))/((Link_Flow_0).sum(axis=1))
    Link_Flow_0 = Link_Flow_2
#error_value=M*(array(k_value*Link_Flow_2.T)[0][0]-K)**2
#M=M*2

0.5393584251029795
0.1054347428293155
0.1492246097588377
0.07135196129346998
0.10835608861992466
0.07212628477981002
0.09246882478361837
0.13170284999991244
0.05095934174219499
0.06973981262807144
0.04211753852504407
0.05884715949542884
0.03601551284197859
0.04930068714036158
0.07521776847428795
0.03202803110080588
0.045442233412307326
0.027912308211729207
0.04000996259161044
0.024765078658794976
0.03577631168552943
0.05036588263405231
0.01628162846411786
0.013280679892716987
0.03366307320474416
0.02091428018076405
0.030293635904751443
0.018914884719754346
0.02747737654334863
0.042751187362241774
0.018817994912705444
0.02731554786938275
0.017013193372608496
0.02481834240760882
0.015520511176607051
0.022733897318078098
0.032646358896193294
0.010860772004271822
0.008949674895952841
0.02261424965310081
0.014155446786603416
0.020729444855958633
0.013016250328200146
0.019197718932608177
0.03020936739534511
0.013477939832726143
0.019739389881301535
0.012360633607749414
0.01816209644905835
0.

0.0017443405251579213
0.0026018725059511107
0.0038634606063835493
0.0013527536136425198
0.00111640969807644
0.002814718465351432
0.0017824520805828892
0.0026578127941555177
0.0016836957722069677
0.0025874499903209775
0.00413707434446172
0.001883744990117827
0.0028079970449350674
0.001777760344738445
0.002651037491644601
0.0016789951139798277
0.0025046618461624613
0.003727136610101391
0.0013209293719522617
0.0010807358901691352
0.002706116926211133
0.0017137726699428763
0.0025557141706149893
0.003508961335335847
0.0011998114714195215
0.0011239431096474067
0.002749820777666578
0.0017413570282551502
0.0025961401337827347
0.0016446162222236097
0.0023088545427939716
0.0039050005412666604
0.0018158717315193507
0.002706664343007775
0.0017138022397455116
0.002555480244403167
0.0016186389503351597
0.0024144250139404824
0.0035472108043176046
0.0011838647577872249
0.0010170599912015642
0.0026298555286186427
0.0016655297327184656
0.0024835279919690357
0.001573387560606937
0.002303992214008729
0.00

0.0006894752633058871
0.0005818026062646477
0.0014943909182449054
0.0009469475049025993
0.0014134368074074444
0.0008958206058442647
0.0013400940861626096
0.0021845832335405787
0.0010028641713996204
0.001496666647741167
0.0009482771895952579
0.0014154908240083194
0.0008970169114873994
0.0013392321670885605
0.0019891705556904656
0.0006879509781922046
0.000573408851756448
0.0014593577514038748
0.0009247644507797184
0.0013803984534434355
0.0008748927768446841
0.001329748906779565
0.00214691634721614
0.0009819813057153457
0.001465570937443126
0.0009285843366710076
0.0013861551964660763
0.0008784316770999408
0.0013115359428470127
0.00195264085323477
0.0006849188682665464
0.0005650584940367072
0.0014266433001319193
0.000904048787191413
0.001349538894521336
0.0008553444784828654
0.0013177494493968273
0.0021102570254956728
0.0009622018032578566
0.0014361121750055574
0.0009099286212972374
0.0013583591143642515
0.0008608223737800368
0.0012852891612060756
0.0019174580323426608
0.000680673963531794

In [14]:
Link_Flow_0

matrix([[801.59657648, 398.40342352, 432.37704162, 367.62295838,
         862.42747519, 371.5461429 , 862.42747519,   0.        ,
         408.73876817, 453.68870702, 807.1421917 , 167.62295838,
         571.5461429 , 167.62295838, 192.8578083 , 428.4538571 ,
           0.        , 398.40342352, 571.5461429 ]])

In [15]:
t_value=multiply(Link_Relation[3,:],mat((array(Link_Flow_0)/array(Link_Relation[2,:]))**4)*0.15+1)

In [16]:
t_value*Link_Flow_0.T

matrix([[77203.40042398]])